---

# **Importing of Libraries**

- In this section, we have imported all the libraries that we will used to scrap ebay products.

---

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

---

# **Ebay Scraper**

- In this section, we will scrap the items as highlighted in the red box in the following image.

![img](./images/ebay-webpage.PNG?raw=true)

- We will use the following user defined function to extract products from ebay website.

---

In [2]:
def get_url(search_item, page):
    """Search the item using the url."""

    init_url = "https://www.ebay.com/sch/i.html?_from=R40&_nkw={}"
    search_item = search_item.replace(" ", "+")
    final_url = init_url.format(search_item)
    final_url = final_url + "&_sacat=0&_pgn={}"
    return final_url.format(page)

def extractRow(item):
    """Extract details of each row of Ebay Product."""

    try:
        URL = item.find('a', {'class': 's-item__link'})['href']
    except AttributeError:
        URL = ''

    try:
        title = item.find('h3', {'class': 's-item__title'}).text.strip()
    except AttributeError:
        title = ''

    try:
        price = item.find('div', {'class': 's-item__detail s-item__detail--primary'}).text
    except AttributeError:
        price = ''

    try:
        product_subtitle = item.find('div', {'class': 's-item__subtitle'}).text
    except AttributeError:
        product_subtitle = ''

    try:
        product_condition = item.find('span', {'class': 'SECONDARY_INFO'}).text
    except AttributeError:
        product_condition = ''

    try:
        shipping_price = item.find('span', {'class': 's-item__shipping s-item__logisticsCost'}).text
    except AttributeError:
        shipping_price = ''

    try:
        product_location = item.find('span', {'class': 's-item__location s-item__itemLocation'}).text
    except AttributeError:
        product_location = ''

    return (title, price, product_subtitle, product_condition, shipping_price, product_location)

records = list()

def main(search, pages):
    """Executes the main program."""

    # Initialize the Chrome driver
    driver = webdriver.Chrome("chromedriver")

    record = list()

    for page in range(1, pages+1):
        driver.get(get_url(search, page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.findAll('div', attrs={'class': 's-item__info clearfix'})[1:]

        for result in results:
            record = extractRow(result)

            if record:
                records.append(record)

    driver.close()

    # # Saving file to a CSV
    data = pd.DataFrame(data=records, columns=['Title', 'Price', 'ProductSubtitle', 'ProductCondition', 'ShippingPrice', 'ProductLocation'])
    data.to_csv('results.csv', index=False)
    return data.head()

In [3]:
# Search and get the data associated with the product
search = input("Enter the Product:")
main(search=search, pages=5)

,Title,Price,ProductSubtitle,ProductCondition,ShippingPrice,ProductLocation
0,Smartphone 4GB+64GB Android 10 Dual SIM 10-Cor...,$86.85 to $88.85,Best Holiday Gift! Cheap Price w/ High Quality...,Brand New,+$39.99 shipping,from United States
1,UNLOCKED EXCELLENT Apple iPhone 4S 16GB Verizo...,$34.49,Pre-Owned,Pre-Owned,+$12.00 shipping,from United States
2,Apple iPhone SE 2nd Gen 2020 64GB 128GB - Unlo...,$245.95 to $275.98,"A excellent condition, 30 day free returns",Pre-Owned,+$75.00 shipping,from United States
3,Samsung Galaxy S8 - Factory Unlocked - T-Mobil...,$120.66,Pre-Owned,Pre-Owned,+$14.49 shipping,from United States
4,OnePlus 7T Pro McLaren 5G Edition T-Mobile UNL...,$280.00,Pre-Owned · OnePlus 7T Pro · 256 GB · T-Mobile,Pre-Owned,+$34.00 shipping,from United States
